<h1 style="display:inline;">Healthcare Deep Learning with TensorFlow</h1>
<br>
<h2 style="display:inline;">Spiro Ganas, MS</h2>


<a href="https://www.kaggle.com/spiroganas/healthcare-deep-learning-table-of-contents">Table of Contents</a> 
<br>
<h2>Chapter 8 - Medical Image Analysis</h2>


<strong>This tutorial will explain how to use a TPU and TRFrecords to train a classification model on medical images.</strong>

An interactive version of the notebook is available on Kaggle at:<br>
https://www.kaggle.com/spiroganas/healthcare-deep-learning-chapter-8
<hr>

Kaggle's [RANZCR CLiP - Catheter and Line Position Challenge](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification) provides data in the [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord).  TensorFlow can quickly read data stored in the TFRecord format.  This is especially important when the model is being trained on fast GPUs or [TPUs](https://cloud.google.com/tpu) (where IO is frequently the training bottleneck).

This notebook shows how to to use TFRecords, TensorFlow and a TPU to train a medical image classification model. 




# Step 1: Determine if you are using a TPU or GPU/CPU

This section sets up your TPU or GPU, and sets some flags so the rest of the code can be optimized for the accelerator you are using.

Setting up the TPU needs to be the first step in the program.

In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Using the TPU :-)")
    print("Number of accelerators: ", strategy.num_replicas_in_sync)
    USE_TPU = True
    USE_GCS = True
    
    # If using a TPU, turn off eager execution
    tf.config.run_functions_eagerly(False)

except:
    USE_TPU = False
    USE_GCS = False




if not USE_TPU:
    # Set up the GPU
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines
    print("Using the GPU...")
    # View information about the GPU
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print("No GPU enabled!")
        print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
        print('and then re-execute this cell.')
    else:
        print(gpu_info)
        print()
        print("------------------------------------------------------------------------")
        print()



# This lets you save to the google drive when running a TPU
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')




In [ ]:
# Constants used to train the model


DATASET_IMAGE_SIZE = 800

IMAGE_SIZE = 240 #512, 750

LEARNING_RATE = 0.001      # use 0.00001 if training the unfrozen model, 0.001 is the default
NUMBER_OF_EPOCHS = 17
STEPS_PER_EPOCH = None
BATCH_SIZE=64
USE_CUSTOM_LOSS_FUNCTION = True

# Step 2: Import libraries

In [ ]:
from io import BytesIO
from imageio import imread
import os
import random
import csv
import glob
import datetime

# clear out anything leftover from the last run
tf.keras.backend.clear_session()

# Step 3: Find the Data

GPUs can access data on your local drives, but TPUs can only read data from Google Cloud Storage (GCS).
Kaggle has created a helper function that will let you read the files from GCS.

We are also using the TFRecord version of the data.  The TFRecord format was designed to maximize the input/output (IO) speed for your model. 



In [ ]:
if USE_GCS:
    # If you are using a TPU, the input data needs to be stored on Google Cloud Storage
    from kaggle_datasets import KaggleDatasets  # Helper function from Kaggle that gives you access to a GCS bucket holding the competition data
    training_data_folder = KaggleDatasets().get_gcs_path("ranzcr-clip-catheter-line-classification") + "/train_tfrecords/"
    !gsutil ls $training_data_folder
else:
    # GPUs can read data that is stored locally or on GCS,
    # but local storage should be faster.
    training_data_folder = '/kaggle/input/ranzcr-clip-catheter-line-classification/train_tfrecords/'
print(training_data_folder)

# Step 4:  Decrease precision to improve speed and memory utilization

TPUs (and some GPUs) can use smaller datatypes (e.g. fewer decimal points) to speed up the calculations while reducing the amount of memory used.  This (usually) won't impact the accuracy of your model.

In [ ]:
# Speed up training by enabling mixed precision data types
#https://www.tensorflow.org/guide/mixed_precision
if False:

    if not USE_TPU:
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        
        tf.keras.mixed_precision.set_global_policy(policy)

        print('Compute dtype: %s' % policy.compute_dtype)
        print('Variable dtype: %s' % policy.variable_dtype)

    else:
        pass
        # I turned off the bfloat because it makes tf.keras.layers.experimental.preprocessing.RandomRotation was throw an exception
        #policy = tf.keras.mixed_precision.Policy('mixed_bfloat16')


# Step 5: Adjust the batch size

When using a TPU, you can usually multiply the batch size by the number of processors on the TPU.
TPUs also often have a lot of memory, so you can sometimes cache the dataset to speed things up.

In [ ]:
if USE_TPU:
    # The TPU has a ton of memory, so it can usually handle dataset caching and 1 batch per TPU core
    BATCH_SIZE = BATCH_SIZE * strategy.num_replicas_in_sync 
    ENABLE_CACHE = True
else:
    ENABLE_CACHE = False

# Step 6: Create a function that loads THE TFRecords into a tf.data.Dataset


https://www.tensorflow.org/guide/data

TFRecord files can be loaded into a tf.data.Dataset.
The dataset's map() method can then be used to preprocess the data.  https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map




Datasets can  can be fed directly into your model's model.fit().  They can also be "optimized" to speed up the training process.

In [ ]:
def create_RANZCR_datasets(
    TRAINING_DATA_FOLDER ,
    DATASET_SIZE = 30083,  # I looked this up in windows explorer
    val_size = 0.05,  # value from 0 to 1 representing the percent of the data put in the validation data set
    ENABLE_CACHE = True,
    INFINITE_DATASET = False,
    USE_TPU = True,
    USE_GCS = False, 
    IMAGE_SIZE = 300,
    BATCH_SIZE = 64,
    SHUFFLE_DATASET = True,
    KEEP_ASPECT_RATIO = True  # keeps the ratio or height to width, by padding with zeros where required
    ):

    print(USE_GCS)

    # If these files can't be read, grant read/write to your gmail address on the google cloud storage bucket
    if USE_GCS:
        train_filenames = tf.io.gfile.glob(TRAINING_DATA_FOLDER + '*.tfrec')
        print("train filenames", train_filenames)
    else:
        train_filenames = glob.glob(TRAINING_DATA_FOLDER + '*.tfrec')



    train_size = int((1.0-val_size) * DATASET_SIZE)
    val_size = int(val_size * DATASET_SIZE)

    # Create a training and a validation datasets
    full_dataset = tf.data.TFRecordDataset(
        train_filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE
    )

    full_dataset = full_dataset.shuffle(buffer_size=31000, seed=37)

    train_dataset = full_dataset.take(train_size)
    val_dataset = full_dataset.skip(train_size).take(val_size)


    # You can enable this if you want to see what a raw record looks like
    # This can help you set up your feature dictionary
    if False:
        for raw_record in train_dataset.take(1):
            example = tf.train.Example()
            example.ParseFromString(raw_record.numpy())
            print(example)
    
    
    
    
    #print("Size of Training Dataset: ", len(list(train_dataset)))
    #print("Size of Validation Dataset: ", len(list(val_dataset)))

    if ENABLE_CACHE and not USE_TPU:
      train_dataset = train_dataset.cache()
      val_dataset = val_dataset.cache()












    # The feature dictionary should describe the data stored in the TFRecord For more details, 
    # see: https://www.tensorflow.org/tutorials/load_data/tfrecord#read_the_tfrecord_file
    feature_dictionary = {
        "StudyInstanceUID": tf.io.FixedLenFeature([], tf.string),
        "ETT - Abnormal": tf.io.FixedLenFeature([], tf.int64),
        "ETT - Borderline": tf.io.FixedLenFeature([], tf.int64),
        "ETT - Normal": tf.io.FixedLenFeature([], tf.int64),
        "NGT - Abnormal": tf.io.FixedLenFeature([], tf.int64),
        "NGT - Borderline": tf.io.FixedLenFeature([], tf.int64),
        "NGT - Incompletely Imaged": tf.io.FixedLenFeature([], tf.int64),
        "NGT - Normal": tf.io.FixedLenFeature([], tf.int64),
        "CVC - Abnormal": tf.io.FixedLenFeature([], tf.int64),
        "CVC - Borderline": tf.io.FixedLenFeature([], tf.int64),
        "CVC - Normal": tf.io.FixedLenFeature([], tf.int64),
        "Swan Ganz Catheter Present": tf.io.FixedLenFeature([], tf.int64),
        "image": tf.io.FixedLenFeature([], tf.string),
    }


    # We need to parse the TFRecords into feature, label pairs.
    # The "features" are the data you are using to make your prediction (i.e. the medical image)
    # The "labels" are what you are trying to predict.
    # We want to transform the TFRecord feature (a string of bytes) back into an image.  
    # Then we want to decode the jpeg image data into a 3-color-channel array, and change it's size.
    # For our labels, we just want to turn them into one long list of zeros and ones.
    
    # Define the first parsing functions that will turn the TFRecord back into an array and a label
    def _parse_function(example, feature_dictionary=feature_dictionary):
        # Parse the input `tf.train.Example` proto using the feature_dictionary.
        # Create a description of the features.
        parsed_example = tf.io.parse_example(example, feature_dictionary)
        return parsed_example


    train_dataset = train_dataset.map(
        _parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    val_dataset = val_dataset.map(
        _parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )




    # Define the second parsing functions that will turn the TFRecord back into an array and a label
    def generate_training_example(example):
        new_image_size = (IMAGE_SIZE, IMAGE_SIZE)

        # Convert the image to an ndarray, resize it and convert it to RGB color
        # These are the settings most commonly required by base models used in transfer learning.


        features = tf.image.decode_jpeg(example["image"], channels=3)  


        if KEEP_ASPECT_RATIO:
            features = tf.image.resize_with_pad(image=features,
                                                target_height=IMAGE_SIZE,
                                                target_width=IMAGE_SIZE,
                                                method=tf.image.ResizeMethod.BILINEAR,
                                                antialias=False
                                                )
        else:
            features = tf.image.resize(features, size=new_image_size)






        labels = [  # Edit this to add whatever labels you want your model to predict
            example["ETT - Abnormal"],
            example["ETT - Borderline"],
            example["ETT - Normal"],
            example["NGT - Abnormal"],
            example["NGT - Borderline"],
            example["NGT - Incompletely Imaged"],
            example["NGT - Normal"],
            example["CVC - Abnormal"],
            example["CVC - Borderline"],
            example["CVC - Normal"],
            example["Swan Ganz Catheter Present"],
        ]

        features = tf.cast(features, tf.float32)
        labels = tf.cast(labels, tf.float32)


        return features, labels


    train_dataset = train_dataset.map(
        generate_training_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.map(
        generate_training_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)




#    if ENABLE_CACHE and USE_TPU:
#        # TPU stores cache in RAM
#        train_dataset = train_dataset.cache()
#        val_dataset = val_dataset.cache()
#    else:
#        # GPU has less memory, so we store the cache on the disk
#        train_dataset = train_dataset.cache("/content/spiro_dataset_cache_train/")
#        val_dataset = val_dataset.cache("/content/spiro_dataset_cache_val/")
        
    if ENABLE_CACHE and USE_TPU:
        train_dataset = train_dataset.cache()
        val_dataset = val_dataset.cache()


    # I don't know if shuffling in required.  It might cause RAM to run out or pre-processing to go very slow
    if SHUFFLE_DATASET:
        train_dataset = train_dataset.shuffle(buffer_size=256, seed=37)



    if INFINITE_DATASET:
        train_dataset = train_dataset.repeat()


    # Set the batch size before applying the data augmentation
    # drop_remainder=True prevents a smaller, last batch.  smaller batches could cause training issues on the TPU (i.e. cause the loss to go to NaN)
    train_dataset = train_dataset.batch(BATCH_SIZE)   #, drop_remainder=True)
    val_dataset = val_dataset.batch(BATCH_SIZE)




#    # Apply data augmentation to the training data set
#    if DATA_AUGMENTATION:
#        data_augmentation = tf.keras.Sequential([
#        tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),                        
#        # Randomly pick about 83% of the area of the image (1/(1.1^2))                                       
#        #tf.keras.layers.experimental.preprocessing.Resizing(height=int(1.1*IMAGE_SIZE), width=int(1.1*IMAGE_SIZE+100), interpolation='bilinear'),
#        #tf.keras.layers.experimental.preprocessing.RandomCrop(height=IMAGE_SIZE, width=IMAGE_SIZE),
#
#        tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1 ),
#
#        tf.keras.layers.experimental.preprocessing.RandomFlip(),
#        tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.2, 0.2), fill_mode='constant'),
#        ], name="Data_Augmentation")
#        
#        train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)


    # Speeds up the pipeline
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)


    return train_dataset, val_dataset








# Step 3:  Create training and validation data sets

In [ ]:
print(training_data_folder)
train_dataset, val_dataset = create_RANZCR_datasets( TRAINING_DATA_FOLDER=training_data_folder,
                                                     DATASET_SIZE = 30083,  # I looked this up in windows explorer
                                                     val_size = 0.10,  # value from 0 to 1 representing the percent of the data put in the validation data set
                                                     IMAGE_SIZE=DATASET_IMAGE_SIZE,
                                                     ENABLE_CACHE = True,
                                                     INFINITE_DATASET = False,
                                                     USE_TPU = USE_TPU,
                                                     USE_GCS = USE_GCS,
                                                     BATCH_SIZE = BATCH_SIZE,
                                                     SHUFFLE_DATASET = True,
                                                     )





print(train_dataset.take(1))
print('-------------------')
for X in train_dataset.take(2):
    print(X[0][0])
    print('-------------------')
    print(X[1][0])

In [ ]:
# List of all the Keras.applications models
# Comment out the ones you don't want to train

# This sets the weights for customized_efficientnet to avoid the exploding gradient problemt
initializer = tf.keras.initializers.HeNormal(seed=42)



    # base model name, Layer to start unfreezing at (None means frozen model, 0 mean train all layers), Keras Model instance, a preprocessing function
Keras_Models = [
#                ["DenseNet121", 0, tf.keras.applications.DenseNet121, tf.keras.applications.densenet.preprocess_input, {'weights':'imagenet', 'include_top':False, 'default_size' : 224,}],
#                ["DenseNet169", 0, tf.keras.applications.DenseNet169, tf.keras.applications.densenet.preprocess_input, {'weights':'imagenet', 'include_top':False, 'default_size' : 224,}],
#                ["DenseNet201", 0, tf.keras.applications.DenseNet201, tf.keras.applications.densenet.preprocess_input, {'weights':'imagenet', 'include_top':False, 'default_size' : 224,}],                
#                ["EfficientNetB0", 0, tf.keras.applications.EfficientNetB0, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 224,}],
                ["EfficientNetB1", 0, tf.keras.applications.EfficientNetB1, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 240,}],
#                ["EfficientNetB2", 0, tf.keras.applications.EfficientNetB2, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 260,}],
#                ["EfficientNetB3", 0, tf.keras.applications.EfficientNetB3, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 300,}],
#                ["EfficientNetB4", 0, tf.keras.applications.EfficientNetB4, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 380,}],
#                ["EfficientNetB5", 0, tf.keras.applications.EfficientNetB5, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 456,}],
#                ["EfficientNetB6", 0, tf.keras.applications.EfficientNetB6, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 528,}],
#                ["EfficientNetB7", 0, tf.keras.applications.EfficientNetB7, tf.keras.applications.efficientnet.preprocess_input, {'weights':'imagenet', 'include_top':False,'drop_connect_rate':0.7, 'default_size' : 600,}],         
#                ["InceptionResNetV2", 0, tf.keras.applications.InceptionResNetV2, tf.keras.applications.inception_resnet_v2.preprocess_input, {'weights':'imagenet', 'include_top':False,}], 
#                ["InceptionV3", 0, tf.keras.applications.InceptionV3, tf.keras.applications.inception_v3.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["MobileNet", 0, tf.keras.applications.MobileNet, tf.keras.applications.mobilenet.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["MobileNetV2", 0, tf.keras.applications.MobileNetV2, tf.keras.applications.mobilenet_v2.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["MobileNetV3Large", 0, tf.keras.applications.MobileNetV3Large, tf.keras.applications.mobilenet_v3.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["MobileNetV3Small", 0, tf.keras.applications.MobileNetV3Small, tf.keras.applications.mobilenet_v3.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["NASNetLarge", 0, tf.keras.applications.NASNetLarge, tf.keras.applications.nasnet.preprocess_input, {'weights':'imagenet', 'include_top':False, 'default_size' : 331,}],
#                ["NASNetMobile", 0, tf.keras.applications.NASNetMobile, tf.keras.applications.nasnet.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["ResNet50", 0, tf.keras.applications.ResNet50, tf.keras.applications.resnet.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["ResNet101", 0, tf.keras.applications.ResNet101, tf.keras.applications.resnet.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["ResNet152", 0, tf.keras.applications.ResNet152, tf.keras.applications.resnet.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["ResNet50V2", 0, tf.keras.applications.ResNet50V2, tf.keras.applications.resnet_v2.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["ResNet101V2", 0, tf.keras.applications.ResNet101V2, tf.keras.applications.resnet_v2.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["ResNet152V2", 0, tf.keras.applications.ResNet152V2, tf.keras.applications.resnet_v2.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["VGG16", 0, tf.keras.applications.VGG16, tf.keras.applications.vgg16.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["VGG19", 0, tf.keras.applications.VGG19, tf.keras.applications.vgg19.preprocess_input, {'weights':'imagenet', 'include_top':False,}],
#                ["Xception", 0, tf.keras.applications.Xception, tf.keras.applications.xception.preprocess_input, {'weights':'imagenet', 'include_top':False,}],                                                                                              
               ]





In [ ]:
def train_model(base_model_name, Start_unfreezing_at_layer, base_model, preprocessing_function, model_arguments, Save_Folder, IMAGE_SIZE):
    """Input is the name of a model, output is a dictionary with the trained modesl name and score"""



    # Keras manages a global state, which it uses to implement the 
    # Functional model-building API and to uniquify autogenerated layer 
    # names. If you are creating many models in a loop, this global state 
    # will  consume an increasing amount of memory over time, and you may 
    # want to  clear it. Calling clear_session() releases the global state:  
    # this helps avoid clutter from old models and layers, especially when  
    # memory is limited.
    #
    # With `clear_session()` called at the beginning,
    # Keras starts with a blank state at each iteration
    # and memory consumption is constant over time.
    # https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session
    tf.keras.backend.clear_session()

    print("Base Model Name: ", base_model_name)
    print("Start Unfreezing at Layer: ", Start_unfreezing_at_layer)
    print("Base Model: ", base_model)
    print("Preprocessing Function: ", preprocessing_function)
    print("Batch Size: ", BATCH_SIZE)

    
#    with strategy.scope():
    tf.keras.backend.clear_session()

    n_labels = 11
    #auc = tf.keras.metrics.AUC(multi_label=True)   


    # if the model has a default size, use that.  otherwise use the IMAGE_SIZE set at the top
    if model_arguments.get('default_size'):
        model_image_size = model_arguments.get('default_size')
        del model_arguments['default_size']
    else:
        model_image_size = IMAGE_SIZE





    # Function for decaying the learning rate.  Need to be inside the strategy.scope()
    # https://stackoverflow.com/questions/56542778/what-has-to-be-inside-tf-distribute-strategy-scope
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_auc", factor=0.2, patience=3, min_lr=1e-9, mode='max',verbose=1)




    def learning_rate_schedule(epoch, lrate):
        if epoch<5:
            lrate = 0.01
        elif epoch<10:
            lrate = 0.01
        else:
            lrate = 0.00001
        return lrate

    learning_rate_schedule_callback = tf.keras.callbacks.LearningRateScheduler(learning_rate_schedule)





    EarlyStopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc', 
        min_delta=0.0001, 
        patience=10, 
        verbose=1,
        mode='max', 
        baseline=None, 
        restore_best_weights=True
    )





#        ModelCheckpoint_folder = CHECKPOINTS_FOLDER +base_model_name+"_ImageSize_" + str(model_image_size) + "/Checkpoint_epoch_{epoch}_auc_{val_auc}.hdf5"  
#        print("Checkpoint files: ", ModelCheckpoint_folder)      
#        ModelCheckpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#                                            filepath = ModelCheckpoint_folder,            
#                                            monitor='val_auc', 
#                                            verbose=1, 
#                                            save_best_only=True,
#                                            save_weights_only=False, 
#                                            mode='max', 
#                                            save_freq='epoch',
#                                            options=save_locally
#        )





    # https://blog.tensorflow.org/2020/04/introducing-new-tensorflow-profiler.html
#        tensorboard_callback = tf.keras.callbacks.TensorBoard(  log_dir = LOG_DIR,
#                                                                profile_batch = '2,7')






#        inputs = tf.keras.layers.Input([IMAGE_SIZE, IMAGE_SIZE, 3])
   # inputs = tf.cast(inputs, tf.float32)
#        x = tf.keras.applications.mobilenet.preprocess_input(x)
    if True:
        # This is my functional code that uses the model from the list above
        inputs = tf.keras.Input(shape=(DATASET_IMAGE_SIZE, DATASET_IMAGE_SIZE, 3))




        resized_inputs = tf.keras.layers.experimental.preprocessing.Resizing(height=model_image_size, width=model_image_size, interpolation='bilinear', name='resize_the_inputs')(inputs)
        preprocessed_inputs = preprocessing_function(resized_inputs)

        # within-model data augmentation
        #preprocessed_inputs = tf.keras.layers.experimental.preprocessing.RandomFlip()(preprocessed_inputs)
        preprocessed_inputs = tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.2, 0.2), fill_mode='constant')(preprocessed_inputs)
        preprocessed_inputs = tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.2 )(preprocessed_inputs)




        # Create an instance of the Base Model
        base_model=base_model( **model_arguments)


        if Start_unfreezing_at_layer is None:
            # Freeze the base model
            for layer in base_model.layers:
                layer.trainable = False
        else:
            # finetuning, unfreeze the top layer of the model
            # If you change the model or want to change the layers that are unfrozen,
            # change the variable Start_unfreezing_at_layer.
            for i, layer in enumerate(base_model.layers):
                if i>=Start_unfreezing_at_layer:
                    layer.trainable=True
                else:
                    layer.trainable=False




        x = base_model(preprocessed_inputs)


        if base_model_name=="EfficientNet_Custom_with_top" or base_model_name=="NFNetF0":
            outputs = x
        else:
            # Add my own top
            # Convert features of shape `base_model.output_shape[1:]` to vectors
            x = tf.keras.layers.GlobalAveragePooling2D()(x)
            # A Dense classifier with a single unit (binary classification)
            outputs = tf.keras.layers.Dense(n_labels, activation='sigmoid')(x)

        model = tf.keras.Model([inputs], [outputs], name=base_model_name)


    else:
        #I've confirmed that this model is identical to the functional model with Start_unfreezing_at_layer=0
        # Testing the model from:
        # https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/204950
        # https://www.kaggle.com/xhlulu/ranzcr-efficientnet-gpu-starter-train-submit
        # B2	260	Val_AUC:0.9206	 	GPU	ImageNet	
        model = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),                         
            tf.keras.applications.EfficientNetB3(
                input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                weights='imagenet',
                include_top=False,
                drop_connect_rate=0.5),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(n_labels, activation='sigmoid')
            ])

    USE_CUSTOM_LOSS_FUNCTION = False
    if USE_CUSTOM_LOSS_FUNCTION:
        from custom_loss_function import custom_loss_fn
        print("Using a custom loss function from an external module!")
        Loss_Function = custom_loss_fn
    else:
        Loss_Function = 'binary_crossentropy'


    LOAD_WEIGHTS_FROM_CHECKPOINT = None
    if LOAD_WEIGHTS_FROM_CHECKPOINT:
        # Loads the weights
        model.load_weights(LOAD_WEIGHTS_FROM_CHECKPOINT)

    # use gradient clipping to prevent loss from going to nan
    # Good default values are clipnorm=1.0 or clipvalue=0.5.
    Custom_Optimizer = tf.keras.optimizers.Nadam(learning_rate=LEARNING_RATE, clipnorm=1.0)
    #Custom_Optimizer = SGD_AGC(lr=LEARNING_RATE)

    model.compile(
        optimizer=Custom_Optimizer,
        loss=Loss_Function,
        metrics=[tf.keras.metrics.AUC(multi_label=True)]
        )

    print(model.summary())



    history = model.fit(
        train_dataset, 
        epochs=NUMBER_OF_EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks=[
                    lr_reducer,
                    #learning_rate_schedule_callback,
                    EarlyStopping_callback,
                #ModelCheckpoint_callback,
                   # tensorboard_callback,
               ],
        validation_data=val_dataset,
    )



    scores = model.evaluate(val_dataset)
    print("Loss: ", scores[0])
    print("Multi-label AUC: ", scores[1])


#        SAVED_MODEL_FOLDER = "/kaggle/working/"
#        filename = (SAVED_MODEL_FOLDER +
#                    base_model_name + 
#                    "_ImageSize_" + str(model_image_size) + 
#                    "_Start_unfreezing_at_layer_" + str(Start_unfreezing_at_layer) + 
#                    "_Epochs_" +str(NUMBER_OF_EPOCHS) + 
#                    "_Multi_label_AUC_" + str(round(scores[1],5)) + 
#                    ".h5")
#        model.save(filename, save_format="h5", options=save_locally)
#        print("Saved the model to: " + filename)




    return [ {  #"filename":  filename,
                "Base Model": base_model_name,
                "Start unfreezing at layer": str(Start_unfreezing_at_layer),
                "Image Size": model_image_size,
                "Initial Learning Rate": LEARNING_RATE,
                "Batch Size": BATCH_SIZE,
                "Loss": scores[0], 
                "Multi-label AUC": scores[1],         
                }]





def Train_and_Save_Models(Keras_Models, Save_Folder, IMAGE_SIZE):
    """Takes in a list of save filenames and models.  Trains the models and then saves them"""
    

    #for_testing


    results = []
    for base_model_name, Start_unfreezing_at_layer, base_model, preprocessing_function, model_arguments in Keras_Models:

        results.append(
                        train_model(base_model_name, Start_unfreezing_at_layer, base_model, preprocessing_function, model_arguments, Save_Folder="/content/drive/MyDrive/ML_DataSets/Catheter_Dataset/Saved_Model/", IMAGE_SIZE=IMAGE_SIZE)
                        )

    return results

try:
    with strategy.scope():
        Results = Train_and_Save_Models(Keras_Models, Save_Folder="/kaggle/working/", IMAGE_SIZE=IMAGE_SIZE)
except IndexError:
    pass  # There was a "pop from empty list" error in "tensorflow/python/distribute/distribution_strategy_context.py" that I'm ignoring

print("Results:  ")
import pprint
pprint.pprint(Results)
print("-----------------------------------------")
print()

